![image info](https://raw.githubusercontent.com/davidzarruk/MIAD_ML_NLP_2023/main/images/banner_1.png)

# Proyecto 1 - Predicción de precios de vehículos usados

En este proyecto podrán poner en práctica sus conocimientos sobre modelos predictivos basados en árboles y ensambles, y sobre la disponibilización de modelos. Para su desasrrollo tengan en cuenta las instrucciones dadas en la "Guía del proyecto 1: Predicción de precios de vehículos usados".

**Entrega**: La entrega del proyecto deberán realizarla durante la semana 4. Sin embargo, es importante que avancen en la semana 3 en el modelado del problema y en parte del informe, tal y como se les indicó en la guía.

Para hacer la entrega, deberán adjuntar el informe autocontenido en PDF a la actividad de entrega del proyecto que encontrarán en la semana 4, y subir el archivo de predicciones a la [competencia de Kaggle](https://www.kaggle.com/t/b8be43cf89c540bfaf3831f2c8506614).

## Datos para la predicción de precios de vehículos usados

En este proyecto se usará el conjunto de datos de Car Listings de Kaggle, donde cada observación representa el precio de un automóvil teniendo en cuenta distintas variables como: año, marca, modelo, entre otras. El objetivo es predecir el precio del automóvil. Para más detalles puede visitar el siguiente enlace: [datos](https://www.kaggle.com/jpayne/852k-used-car-listings).

In [ ]:
!pip install lightgbm

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Importación librerías
%matplotlib inline
import pandas as pd
import numpy as np
from random import randrange
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import StandardScaler

from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error, mean_absolute_error

from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import BaggingRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor

import datetime

In [ ]:
# Carga de datos de archivo .csv
dataTraining = pd.read_csv('https://raw.githubusercontent.com/davidzarruk/MIAD_ML_NLP_2023/main/datasets/dataTrain_carListings.zip')
dataTesting = pd.read_csv('https://raw.githubusercontent.com/davidzarruk/MIAD_ML_NLP_2023/main/datasets/dataTest_carListings.zip', index_col=0)

In [ ]:
# Visualización datos de entrenamiento
dataTraining.head()

In [ ]:
dataTraining['Make'] = dataTraining['Make'].astype('category')
dataTraining['Model'] = dataTraining['Model'].astype('category')
dataTraining['State'] = dataTraining['State'].astype('category')
dataTraining.dtypes

In [ ]:
dataTraining.isnull().sum()

In [ ]:
dataTraining.shape

### Exploración de datos antes del preprocesamiento

In [ ]:
dataTraining.describe()

In [ ]:
dataTraining.shape

In [ ]:
conteo_make = dataTraining['Make'].value_counts()
conteo_make.plot.bar(figsize=(10,6), fontsize=12, color='blue')
plt.title('Cantidad de marcas de autos', fontsize=16)
plt.xlabel('Marca', fontsize=14)
plt.ylabel('Cantidad', fontsize=14)
plt.show()

In [ ]:
MatrixCorrelacion = dataTraining.corr()

fig, ax = plt.subplots(figsize=(10, 5))
sns.set(font_scale=1.5)  

sns.heatmap(MatrixCorrelacion, annot=True, ax=ax)
plt.show()

In [ ]:
# Histograma para todas las variables numéricas incluyendo la variable de respuesta
dataTraining.hist(bins=50, figsize=(20,15))
plt.tight_layout()
plt.show()

In [ ]:
plt.scatter(dataTraining.Year,dataTraining.Price)
plt.xlabel("Year")
plt.ylabel("Price")
plt.show()

In [ ]:
plt.scatter(dataTraining.Mileage,dataTraining.Price)
plt.xlabel("Mileage")
plt.ylabel("Price")
plt.show()

In [ ]:
# Agrupamos por Estado y calculamos el promedio de los precios
data = dataTraining.groupby('State')['Price'].mean()

# Ordenamos los valores de mayor a menor
data = data.sort_values(ascending=False)

# Creamos la gráfica de barras
plt.figure(figsize=(15,10))
plt.bar(data.index, data.values)
plt.xticks(rotation=90)
plt.xlabel('State')
plt.ylabel('Precio Promedio')
plt.title('Promedio de precios por estado')
plt.show()

### Preprocesamiento de datos

#### Edad del vehículo

In [ ]:
time_now = datetime.datetime.now()
dataTraining['Age'] = dataTraining['Year'].apply(lambda x : time_now.year - x)
dataTraining = dataTraining.drop(['Year'], axis=1)

#### Eliminación de duplicados

In [ ]:
# Eliminar duplicados
duplicados = dataTraining.duplicated()
print("Número total de duplicados en el DataFrame: ", duplicados.sum())

In [ ]:
dataTraining = dataTraining.drop_duplicates()

In [ ]:
dataTraining.head()

#### Eliminación de valores atípicos

In [ ]:
# VALORES ATÍPICOS PARA MILEAGE
# Calcular el rango intercuartil (IQR)
Q1 = dataTraining['Mileage'].quantile(0.25)
Q3 = dataTraining['Mileage'].quantile(0.75)
IQR = Q3 - Q1

# Eliminar los valores atípicos utilizando el método del rango intercuartil (IQR)
dataTraining = dataTraining[~((dataTraining['Mileage'] < (Q1 - 1.5 * IQR)) | (dataTraining['Mileage'] > (Q3 + 1.5 * IQR)))]

In [ ]:
dataTraining.head()

In [ ]:
plt.figure(figsize=(8, 6))
plt.boxplot(dataTraining['Mileage'], vert=False)
plt.title('Boxplot de la variable "mileage"', fontsize=16)
plt.xlabel('Mileage', fontsize=14)
plt.yticks(fontsize=12)
plt.show()

#### Estandarización Mileage en data Training y Testing

In [ ]:
# Fit transform

scaler = StandardScaler()
scaler.fit(dataTraining[['Mileage']])
dataTraining[['Mileage']] = scaler.transform(dataTraining[['Mileage']])
dataTesting[['Mileage']] = scaler.transform(dataTesting[['Mileage']])

#### Dummies

##### Marca

In [ ]:
# Eliminar espacios en la columna "Make"
dataTraining["Make"] = dataTraining["Make"].apply(lambda x: x.strip())

############################## Dummies para Marca ####################################
dummies = pd.get_dummies(dataTraining['Make'], prefix='Make')
dummies=dummies.drop('Make_Freightliner', axis=1)
dataTraining = pd.concat([dataTraining, dummies], axis=1)

##### Modelo
En el caso de la variable modelo, se crea una variable dummie teniendo en cuenta la media del precio para un modelo específico. Después se mapean esas particiones para asignar la misma dummie en la data de testing.

In [ ]:
######################## Dummies para los modelos ##########################

# Crear un diccionario vacío para almacenar los resultados
promedios_por_modelo = {}

# Recorrer los valores distintos de la columna "Model"
for model in dataTraining["Model"].unique():
    # Obtener el promedio de la columna "Price" para los registros donde "Model" es igual a la marca actual
    promedio = dataTraining.loc[dataTraining["Model"] == model, "Price"].mean()
    # Agregar la marca y su promedio al diccionario
    promedios_por_modelo[model] = promedio
diccionario_ordenado = dict(sorted(promedios_por_modelo.items(), key=lambda x: x[1], reverse=True))

In [ ]:
k = 100
column_names = []
for i in range(0, 80000, int(80000/k)):
    name = f"{i}-{i+int(80000/k)}"
    column_names.append(name)

# Creamos las columnas con el nombre de la partición en el dataframe
for i in range(len(column_names)):
    dataTraining[column_names[i]]=0

# Creamos diccionario con el rango de precios como llaves y los modelos que se encuentran en ese rango como valores
rango_precios = {}

for column in column_names:
    start, end = column.split("-")
    start, end = int(start), int(end)
    rango_precios[column] = [key for key, value in diccionario_ordenado.items() if start <= value <= end]

# Validar si el valor de la columna Model está en el diccionario rango_precios
for i, row in dataTraining.iterrows():
    model = row['Model']
    for key, value in rango_precios.items():
        if model in value:
            dataTraining.at[i, key] = 1

##### Estado

In [ ]:
# Eliminar espacios en la columna "State"
dataTraining["State"] = dataTraining["State"].apply(lambda x: x.strip())

############################## Dummies para State ####################################
dummies = pd.get_dummies(dataTraining['State'], prefix='State')
dataTraining = pd.concat([dataTraining, dummies], axis=1)

dataTraining = dataTraining.drop(['State', 'Make', 'Model'], axis=1)

#### Transformación variable "Price"
En la gráfica a continuación, se puede observar que la distribución de la variable "Price" es asimétrica hacia la derecha. Por lo tanto, para la construcción del modelo se transforma la variable de respuesta con el logaritmo y después se aprecia que la distribución de la variable es casi normal.

In [ ]:
from scipy.stats import norm
fig, ax = plt.subplots(1,2)
width, height = fig.get_size_inches()
fig.set_size_inches(width*2, height)
sns.distplot(dataTraining['Price'], ax=ax[0], fit=norm)
sns.distplot(np.log(dataTraining[('Price')]+1), ax=ax[1], fit= norm)

In [ ]:
dataTraining['Price'] = np.log1p(dataTraining['Price'])

#### Separación de datos en set de entrenamiento y test

In [ ]:
# Separación de variables predictoras (X) y variable de interés (y)
y = dataTraining['Price']
X = dataTraining.drop(['Price'], axis=1)

# Separación de datos en set de entrenamiento y test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

### Calibración del modelo

In [ ]:
nEstimators = 350
nSubsample = 0.8
nDepth = 10
nLearningRate = 0.3
nGamma = 0
nColsample = 0.9

### Entrenamiento del modelo

#### XGBoost

In [ ]:
MODELO = XGBRegressor(n_estimators=nEstimators, subsample=nSubsample, max_depth=nDepth, learning_rate=nLearningRate,
                        gamma=nGamma, colsample_bytree=nColsample, random_state=12345, n_jobs=-1)
MODELO.fit(X_train, y_train)

In [ ]:
#Impresión de desempeño del modelo
y_pred = MODELO.predict(X_test)
y_pred

In [ ]:
#y_test = np.expm1(y_test)
y_pred = np.expm1(y_pred)

In [ ]:
# RMSE
xgb_RMSE = mean_squared_error(y_test, y_pred, squared=False)
print("RMSE: %.3f" %xgb_RMSE )
# MAE
xgb_MAE = mean_absolute_error(y_test, y_pred)
print("MAE: %.3f" %xgb_MAE )

### Transformación de variables en testing

In [ ]:
dataTesting = pd.read_csv('https://raw.githubusercontent.com/davidzarruk/MIAD_ML_NLP_2023/main/datasets/dataTest_carListings.zip', index_col=0)
dataTesting['Age'] = dataTesting['Year'].apply(lambda x : time_now.year - x)
dataTesting = dataTesting.drop(['Year'], axis=1)

In [ ]:
# Fit transform

scaler = StandardScaler()
scaler.fit(dataTraining[['Mileage']])
dataTraining[['Mileage']] = scaler.transform(dataTraining[['Mileage']])
dataTesting[['Mileage']] = scaler.transform(dataTesting[['Mileage']])

In [ ]:
########################### Dummies marca ########################################
dummies = pd.get_dummies(dataTesting['Make'], prefix='Make')
dataTesting = pd.concat([dataTesting, dummies], axis=1)

########################### Dummies modelo #######################################

# Creamos las columnas con el nombre de la partición en el dataframe

for i in range(len(column_names)):
    dataTesting[column_names[i]]=0

# Validar si el valor de la columna Model está en el diccionario rango_precios
for i, row in dataTesting.iterrows():
    model = row['Model']
    for key, value in rango_precios.items():
        if model in value:
            dataTesting.at[i, key] = 1

############################## Dummies para State ####################################
dummies = pd.get_dummies(dataTesting['State'], prefix='State')
dataTesting = pd.concat([dataTesting, dummies], axis=1)
            
dataTesting=dataTesting.drop(['State', 'Make','Model'], axis=1)

### Resultados

In [ ]:
y_pred=MODELO.predict(dataTesting)
y_pred = np.expm1(y_pred)

In [ ]:
# Guardar predicciones en formato exigido en la competencia de kaggle
y_pred.to_csv('test_submission.csv', index_label='ID')
y_pred.head()